In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

crime = 'C:/Users/brkic/OneDrive/Desktop/fondamenti/progetto/U.S.Executions.csv'
df = pd.read_csv(crime)

# Essendo che dopo la colonna 14 troviamo solo il numero di vittime per ogni 'razza',
# creo una nuova tabella con i dati a cui sono più interesata
df1 = df.iloc[:, :14]

#Rinomino alcune colonne
df1 = df1.rename(columns={'Execution Volunteer': 'Execution', 'Number of Victims': 'NumV', 'Execution Date': 'ExecutDay'})


'''DOMANDA_1: Qualè il numero di omicidi/esecuzioni tra maschi o femmine di ogni etnia?'''

# Iommare il numero di omicidi compiuti in base a etnia e sesso
omicide = df1.groupby(['Race', 'Sex'])['NumV'].sum().reset_index()
omicide = omicide.drop([5, len(omicide)-1])

# Ier creare piu facilmente il grafico preparo i dati in questa tabela metendo come indici le
# etnie, solone il genere, e i valori sono il numero di omicidi compiuti 
grouped = omicide.pivot_table(index='Race', columns='Sex', values='NumV', aggfunc=np.sum, fill_value=0)

# Preparare i dati per il plottaggio 3D
# estrae gli indici (razze) e le colonne (sessi) dal DataFrame pivot 'grouped'
races = grouped.index
sexes = grouped.columns

# Array di posizioni per x=razze e y=sessi
x = np.arange(len(races))
y = np.arange(len(sexes))

# creiamo coordinate matricali usando x, y (le rendiamo in 2D)
xpos, ypos = np.meshgrid(x, y, indexing="ij")

# Appiattisci le coordinate (le rimettiamo in 1D)
x3d = xpos.flatten()
y3d = ypos.flatten()
z3d = np.zeros_like(x3d)

# Altezze delle barre, che rapresenta l'asse z
dz = grouped.values.flatten()

# Imposta lo stile
sns.set(style='whitegrid', font_scale=1.2)

# Crea la figura 3D
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Colori per i sessi
colors = {'Male': '#4682B4', 'Female': '#FF6347'}  # blu per maschi, rosso per femmine

# Dimensioni delle barre
dx = dy = 0.8

# Crea le barre 3D con trasparenza e aggiungi i numeri sopra di esse
for i in range(len(dz)):
    if y3d[i] == 0:  # Colonne davanti (Femmine)
        ax.bar3d(x3d[i], y3d[i], z3d[i], dx, dy, dz[i], color=colors['Female'], zsort='average', alpha=0.6)
    else:  # Colonne dietro (Maschi)
        ax.bar3d(x3d[i], y3d[i], z3d[i], dx, dy, dz[i], color=colors['Male'], zsort='average', alpha=0.6)

# Etichette e titolo
ax.set_zlabel('Numero di omicidi', fontsize=14)
ax.set_title('Numero di omicidi per razza e sesso', fontsize=16, weight='bold')

# Impostiamo x e y
ax.set_xticks(x)
ax.set_xticklabels(races, rotation=45, ha='right')
ax.set_yticks(y)
ax.set_yticklabels(sexes)

# Mostra il plot
plt.tight_layout()
plt.show()

'''ESECUZIONI'''

# Modifichiamo Execution ponendo i valori 'yes' in True e 'no' in False
df1['Execution'] = df1['Execution'].map({'yes': True, 'no': False})

# Contiamo il numero totale di persone giustiziate per etnia e sesso
execution = df1[df1['Execution'] == True].groupby(['Race', 'Sex']).size().reset_index(name='NumE')

# Per creare piu facilmente il grafico preparo i dati in questa tabela metendo come indici le
# etnie, solone il genere, e i valori sono il numero di omicidi compiuti
groupedE = execution.pivot_table(index='Race', columns='Sex', values='NumE', aggfunc=np.sum, fill_value=0)

# Calcoliamo le percentuali di esecuzioni rispetto al totale
total_by_race_sex = groupedE.sum().sum()
percentages = (groupedE / total_by_race_sex) * 100


# Crea la figura e gli assi del grafico
fig, ax = plt.subplots(figsize=(12, 8))

# Impostare quanto le barre saranno distanti
bar_width = 0.4

# Posizioni per le barre sull'asse x
x = range(len(groupedE.index))

# Barre per i maschi (colore blu)
bars_male = ax.bar(x, groupedE['Male'], width=bar_width, color=colors['Male'], label='Maschi')

# Barre per le femmine (colore rosso, spostate a destra di bar_width)
bars_female = ax.bar([pos + bar_width for pos in x], groupedE['Female'], width=bar_width, color=colors['Female'], label='Femmine')

# Impostare la x = posiziona le etichette al centro delle coppie di barre
ax.set_xticks([pos + bar_width / 2 for pos in x])
ax.set_xticklabels(groupedE.index, rotation=45, ha='right')

# Etichette e titolo del grafico
ax.set_xlabel('Etnia', fontsize=14)
ax.set_ylabel('Numero di Esecuzioni', fontsize=14)
ax.set_title('Numero di Esecuzioni per Etnia e Sesso', fontsize=16, weight='bold')

ax.legend(fontsize=12)

# Aggiungi le annotazioni sopra le barre con il numero di esecuzioni
for bars in [bars_male, bars_female]:  
    for bar in bars:  
        height = bar.get_height()
        ax.annotate(f'{height}', 
                    xy=(bar.get_x() + bar.get_width() / 2, height),  
                    xytext=(0, 3),  
                    textcoords="offset points", 
                    ha='center', va='bottom',  
                    fontsize=10, color='black', weight='bold')  

# Personalizzazione dello stile dei ticks e delle griglie
ax.tick_params(axis='both', which='major', labelsize=12)  # Stile dei ticks
ax.grid(axis='y', linestyle='--', alpha=0.7)  # Griglia orizzontale con stile a tratteggio e opacità

# Rimozione dei bordi del grafico
sns.despine()

# Aggiungi una sfumatura di colore di sfondo
ax.set_facecolor('#F5F5F5')

# Mostra il grafico finale
plt.tight_layout()
plt.show()

'''DOMANDA_2: Il rapporto tra Assassini/Esecuzioni tra le varie etnie e proporzionato'''

# Troviamo tutte le razze presenti nel DataFrame df1
all_races = df1['Race'].unique()

# DataFrame per omicidi e esecuzioni con tutte le razze
omicide_race = pd.DataFrame({'Race': all_races})
execution_race = pd.DataFrame({'Race': all_races})

# Tabella per gli omicidi
omicide_race = df1.groupby('Race')['NumV'].sum().reset_index(name='NumV')
# Numero di assasini
omicide_race['NumA'] = df1.groupby('Race').size().values
omicide_race = omicide_race.drop([len(omicide_race)-1])

# Calcolo le percentuali di omicidi
total_omicidi = np.sum(omicide_race['NumV'])
omicide_race['PercentOmicidi'] = (omicide_race['NumV'] / total_omicidi) * 100

# Tabella per le esecuzioni
execution_race = df1[df1['Execution'] == True].groupby('Race').size().reset_index(name='NumE')

# Calcolo le percentuali di esecuzioni
total_executions = np.sum(execution_race['NumE'])
execution_race['PercentExecutions'] = (execution_race['NumE'] / total_executions) * 100

# Uniamo entrambe le tabelle per avere lo stesso numero di righe
merged_race = pd.merge(omicide_race, execution_race, on='Race', how='outer')

# Sostituire i valori NaN in 'NumE' con 0
merged_race['NumE'].fillna(0, inplace=True)

# Sostituiamo i valori NaN in 'PercentExecutions' con 0
merged_race['PercentExecutions'].fillna(0, inplace=True)

# Calcolo il rapporto tra il numero di assassini e il numero di giustiziati per ogni razza
merged_race['Ratio'] = (merged_race['NumA'] / merged_race['NumE']).replace([np.inf, -np.inf, np.nan], 0)

# Calcolo le percentuali del numero di assassini per razza
total_assassins = np.sum(merged_race['NumA'])
merged_race['PercentAssassins'] = (merged_race['NumA'] / total_assassins) * 100


skin_tone_colors = {
    'White': 'lightcoral',
    'Black': 'dimgray',
    'Asian': 'yellow',
    'Latinx': 'Peru',
    'Other': 'gray',
    'American Indian or Alaska Native': 'SaddleBrown'
}

# L'indice della fetta con la percentuale più alta di omicidi rispetto alle esecuzioni
max_ratio_index = merged_race['Ratio'].idxmax()

# Esplosioni di tutte le fette tranne quella con la percentuale più alta esplosa
explode = [0.1 if i == max_ratio_index else 0 for i in range(len(merged_race))]

# Filtra le percentuali inferiori al 0.5% 
labels_filtered = []
percentages_filtered = []
explode_filtered = []

# Filtrare le percentuali e preparare le etichette, percentuali e esplosioni filtrate
for i, percent in enumerate(merged_race['PercentOmicidi']):
    if percent >= 0.5 or i == max_ratio_index:
        labels_filtered.append(merged_race['Race'][i])
        percentages_filtered.append(percent)
        explode_filtered.append(explode[i])

# Creazione della figura e degli assi per i pie charts
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Torta - Rapporto Omicidi/Esecuzioni
axes[0].pie(merged_race['Ratio'], labels=merged_race['Race'], autopct=lambda pct: '' if pct < 1 else '%1.1f%%' % pct,
            startangle=140, colors=[skin_tone_colors.get(race, 'gray') for race in merged_race['Race']],
            explode=explode, textprops={'fontsize': 10}, shadow=True)
axes[0].set_title('Rapporto Omicidi/Esecuzioni', fontsize=14)

#Torta - Percentuale di Omicidi per razza
axes[1].pie(percentages_filtered, labels=labels_filtered, autopct=lambda pct: '' if pct < 1 else '%1.1f%%' % pct,
            startangle=140, colors=[skin_tone_colors.get(race, 'gray') for race in labels_filtered],
            explode=explode_filtered, textprops={'fontsize': 10}, shadow=True)
axes[1].set_title('Percentuale di Omicidi', fontsize=14)

# Mostra i pie charts
plt.tight_layout()
plt.show()

'''DOMANDA_3: Durante gli anni il numero di omicidi ed esecuzioni e calato? 
        (sotto domanda) - qurante gli anni qualè l'etnia con maggiori vittime'''

# Convertire 'Execution Date' in datetime e estrarre l'anno
df1['ExecutDay'] = pd.to_datetime(df1['ExecutDay'], errors='coerce', format='%m/%d/%y')  # Coerce per impostare le date non valide a NaT
df1['Year'] = df1['ExecutDay'].dt.year

# Calcolo il numero di assasini per anno
assassinYear = df1.groupby(df1['Year']).size().reset_index(name='NumAssasin')

# Sommare il numero di giustiziati per ciascun anno
executionYear = df1[df1['Execution']].groupby(['Year']).size().reset_index(name='NumExecutie')

# Uniamo i risultati
yearOmicideExecut= pd.merge(assassinYear, executionYear, on='Year', how='outer').fillna(0)

#--------------------------------------------------------------------------------------------------

#Riduciamo e rinominominiamo le colonne per semplicità
df.rename(columns={
    'Number of White Male Victims': 'WhiteM',
    'Number of Black Male Victims': 'BlackM',
    'Number of Latino Male Victims': 'LatinxM',
    'Number of Asian Male Victims': 'AsianM',
    'Number of Native American Male Victims': 'American Indian or Alaska NativeM',
    'Number of Other Race Male Victims': 'OtherM',
    'Number of White Female Victims': 'WhiteF',
    'Number of Black Female Victims': 'BlackF',
    'Number of Latino Female Victims': 'LatinxF',
    'Number of Asian Female Victims': 'AsianF',
    'Number of American Indian or Alaska Native Female Victims': 'American Indian or Alaska NativeF',
    'Number of Other Race Female Victims': 'OtherF',
    'Execution Date': 'ExecutDay'
}, inplace=True)

# Converte la colonna 'ExecutDay' in datetime e ottiene l'anno
df['ExecutDay'] = pd.to_datetime(df['ExecutDay'], errors='coerce', format='%m/%d/%y')
df['Year'] = df['ExecutDay'].dt.year #-++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


ethnicities = ['White', 'Black', 'Latinx', 'Asian', 'American Indian or Alaska Native', 'Other']
victims = pd.DataFrame()


for ethnicity in ethnicities:
    # Selezioniamo le colonne corrispondenti a maschi e femmine per l'etnia specifica
    male_col = f'{ethnicity}M'
    female_col = f'{ethnicity}F'
    
    # Totale delle vittime (somma di maschi e femmine) per ogni anno
    total_victims = df.groupby('Year')[[male_col, female_col]].sum().sum(axis=1).rename(f'Total {ethnicity} Victims')
    
    # Aggiungiamo la serie delle vittime totali al DataFrame principale
    victims[f'Total {ethnicity} Victims'] = total_victims

# Reset dell'indice per assicurarsi che 'Year' sia una colonna
victims = victims.reset_index()

#--------------------------------------------------------------------------------

# Lista di etnie da plottare
colors = ['lightcoral', 'black', 'yellow', 'orange', 'brown', 'pink']


plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")  # Stile del grafico

# Grafico a linee per il numero di assasini
sns.lineplot(data=yearOmicideExecut, x='Year', y='NumAssasin', label='Assasini', color='red')

# Grafico a linee per il numero di esecuzioni
sns.lineplot(data=yearOmicideExecut, x='Year', y='NumExecutie', label='Esecuzioni', color='gray')

# Aggiungiamo i grafici a linee per le vittime
for ethnicity, color in zip(ethnicities, colors):
    col_male = f'Total {ethnicity} Victims'
    col_female = f'Total {ethnicity} Victims'  # considerando solo le vittime totali per semplicità
    sns.lineplot(data=victims, x='Year', y=col_male, label=f'Vittime {ethnicity}', color=color)

# Titolo 
plt.title('Numero di assasini, esecuzioni e vittime per anno')
plt.xlabel('Anno')
plt.ylabel('Numero Assasini/Esecuzioni/Vittime per anno')

# Sull'asse x ruota ogni anno di 45 gradi 
plt.xticks(yearOmicideExecut['Year'], rotation=45)

# Trova il massimo valore per l'asse y e arrotondalo per eccesso a un multiplo di 10, 
max_value = max(yearOmicideExecut[['NumAssasin', 'NumExecutie']].max().max(), victims.drop(columns='Year').max().max())
plt.yticks(np.arange(0, max_value + 10, 10))

# Creiamo la legenda facendo corrispondere i nomi ai colori
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles, labels, loc='upper left')

# Regola automaticamente le dimensioni del layout per distanziare le etichette sull'asse x
plt.tight_layout()

# Mostra il grafico
plt.show()

#vittime divise tra meschi e femmine-----------------------------------------------------------------------

# Dizionario per salvare le somme
sums_dict = {}

# Togli il grafico che esce in piu e separa le fette 
percentages = pd.DataFrame()


for ethnicity in ethnicities:
    # Somma delle righe della colonna per ogni etnia e sesso
    sum_values = df[f'{ethnicity}M'].sum()  
    sums_dict[f'{ethnicity} Male'] = sum_values
    
    sum_values = df[f'{ethnicity}F'].sum()  
    sums_dict[f'{ethnicity} Female'] = sum_values

totalV = df['Number of Victims'].sum()

# Calcolo del risultato per ogni etnia
for ethnicity in ethnicities:
    # Somma delle vittime maschili e femminili per ogni etnia
    total_male = df[f'{ethnicity}M'].sum()
    total_female = df[f'{ethnicity}F'].sum()
    
    # Calcolo della percentuale per maschi e femmine
    percentage_male = total_male / totalV * 100
    percentage_female = total_female / totalV * 100
    
    # Salvataggio dei risultati in un DataFrame
    percentages.loc[f'{ethnicity}', 'Male Percentage'] = percentage_male
    percentages.loc[f'{ethnicity}', 'Female Percentage'] = percentage_female


# Personalizzismo il testo delle etichete
def my_autopct(pct):
    return f"{pct:.1f}%" if pct >= 1 else ''

# Crea due grafici a torta affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Aggiunta di un piccolo spazio tra le fette
explode = [0.1] * len(ethnicities)  

# Aggiungiamo il colore del bordo e larghezza del bordo
wedgeprops = {'edgecolor': 'black', 'linewidth': 1}

# Torta - Percentuali di vittime maschili per etnia
wedges, texts, autotexts = axes[0].pie(percentages['Male Percentage'], labels=None, autopct=my_autopct, startangle=140,
                                      colors=[skin_tone_colors.get(ethnicity, 'gray') for ethnicity in ethnicities],
                                      explode=explode, wedgeprops=wedgeprops, textprops={'fontsize': 10}, shadow=True)
axes[0].set_title('Percentuali di vittime maschili per etnia', fontsize=14)

# Torta - Percentuali di vittime femminili per etnia
axes[1].pie(percentages['Female Percentage'], labels=None, autopct=my_autopct, startangle=140,
            colors=[skin_tone_colors.get(ethnicity, 'gray') for ethnicity in ethnicities], explode=explode,
            wedgeprops=wedgeprops, textprops={'fontsize': 10}, shadow=True)
axes[1].set_title('Percentuali di vittime femminili per etnia', fontsize=14)

# Aggiunta di una legenda con il colore e il nome delle etnie
legend_labels = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=skin_tone_colors[ethnicity], markersize=10, label=ethnicity) for ethnicity in ethnicities]
fig.legend(handles=legend_labels, labels=ethnicities, loc='center right', fontsize=12)

# Mostra i grafici
plt.tight_layout()
plt.show()


'''DOMANDA_4: Dove vengono comessi piu omicidi'''

state_abbr = {
        'Alabama': 'AL', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 
        'Connecticut': 'CT', 'Delaware': 'DE', 'Federal': 'FED', 'Florida': 'FL', 'Georgia': 'GA', 
        'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Kentucky': 'KY', 'Louisiana': 'LA', 
        'Maryland': 'MD', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 
        'Nevada': 'NV', 'New Mexico': 'NM', 'North Carolina': 'NC', 'Ohio': 'OH', 'Oklahoma': 'OK', 
        'Oregon': 'OR', 'Pennsylvania': 'PA', 'South Carolina': 'SC', 'South Dakota': 'SD', 
        'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Virginia': 'VA', 'Washington': 'WA', 
        'Wyoming': 'WY'
    }

# Mappare i nomi degli stati con le sigle
df1['State'] = df1['State'].map(state_abbr)

# Raggruppo per stato e calcolo il numero totale di omicidi
homicide_counts = df1.groupby('State')['NumV'].sum().reset_index(name='Homicide Count')

# Raggruppo per stato e calcolo il numero totale di esecuzioni
executio = df1[df1['Execution'] == True].groupby('State').size().reset_index(name='Execution Count')

# Unisco i due DataFrame per stato
state_counts = pd.merge(homicide_counts, executio, on='State', how='outer').fillna(0)


# Impostazioni di stile
sns.set(style="whitegrid", palette="pastel")  # Imposta lo stile di Seaborn e la palette pastello

# Grafico a barre affiancate
fig, ax = plt.subplots(figsize=(12, 8))

# Posizioni delle barre
x = np.arange(len(state_counts))  # Utilizza np.arange invece di range per creare un array di interi

# Impostare quanto le barre saranno distanti
bar_width = 0.4

# Grafico delle barre di esecuzioni
bars2 = ax.bar(x, state_counts['Execution Count'], width=bar_width, label='Esecuzioni', color='orange', edgecolor='black')

# Grafico delle barre di omicidi
bars1 = ax.bar(x + bar_width, state_counts['Homicide Count'], width=bar_width, label='Omicidi', color='blue', edgecolor='black')

# Etichette dell'asse
ax.set_xticks(x + bar_width / 2)
ax.set_xticklabels(state_counts['State'], rotation=45, ha='right', fontsize=12)

# Etichette e titolo 
ax.set_xlabel('Stato', fontsize=14, fontweight='bold')
ax.set_ylabel('Numero', fontsize=14, fontweight='bold')
ax.set_title('Numero di Omicidi ed Esecuzioni per Stato', fontsize=16, fontweight='bold')

ax.legend(fontsize=12)

# Aggiunta delle etichette sopra le barre
def add_labels(bars):
    for bar in bars:
        yval = bar.get_height()
        if yval >= 4:  # Mostra l'etichetta solo se il valore è maggiore o uguale a 4
            ax.text(bar.get_x() + bar.get_width() / 2, yval + 2, round(yval, 1), ha='center', va='bottom', fontsize=10, fontweight='bold')

add_labels(bars1)
add_labels(bars2)

# Rimozione delle linee degli assi
sns.despine()

# Mostra il grafico
plt.tight_layout()
plt.show()

#sotto domanda - in ogni stato quante motri per etnia ci sono 

# Creazione di un DataFrame per le vittime per etnia e stato
victimsE = pd.DataFrame()

for ethnicity in ethnicities:
    male_col = f'{ethnicity}M'
    female_col = f'{ethnicity}F'
    
    # Calcolo il totale delle vittime (somma di maschi e femmine) per ogni stato ed etnia
    victimsE[f'{ethnicity}'] = df.groupby('State')[[male_col, female_col]].sum().sum(axis=1)

# Aggiunta le abbreviazioni degli stati nel DataFrame
victimsE['State_abbr'] = victimsE.index.map(state_abbr.get)

# Reset dell'indice per avere 'State' come colonna
victimsE = victimsE.reset_index()

# Calcolo del totale delle vittime per stato
victimsE['Total'] = victimsE[ethnicities].sum(axis=1)

# Calcolo delle percentuali per ogni etnia
for ethnicity in ethnicities:
    victimsE[f'{ethnicity}_perc'] = (victimsE[f'{ethnicity}'] / victimsE['Total']) * 100

# Grafico a pila
fig, ax = plt.subplots(figsize=(12, 8))

bottom = None
bar_width = 0.9  # Aumenta la larghezza delle barre

# Iterazione su ogni etnia per aggiungere le barre
for ethnicity in ethnicities:
    heights = victimsE[f'{ethnicity}_perc']
    bars = ax.bar(victimsE['State'], heights, label=ethnicity, bottom=bottom, width=bar_width)
    if bottom is None:
        bottom = heights
    else:
        bottom = bottom + heights

    # Aggiunta delle percentuali sulle barre
    for i, (bar, height) in enumerate(zip(bars, heights)):
        if height >= 1:  # Condizione per non stampare percentuali inferiori a 1%
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_y() + height / 2,
                    f'{height:.0f}%', ha='center', va='center', color='black', fontsize=8, fontweight='bold')

# Etichette, titoli e legenda
ax.set_xlabel('State')
ax.set_ylabel('Percentage of Victims')
ax.set_title('Percentage of Victims by Ethnicity and State')
ax.legend(title='Ethnicity', loc='lower right')

# Mostra il grafico
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()
